In [3]:
import pandas as pd
import numpy as np
import requests

In [4]:
yearly = True
response = requests.get(f'http://localhost/api/v1/scfa/balancesheet?yearly={yearly}')
balance_sheet = response.json()
balance_sheet

{'2000': {'BBC': {'cash': 8,
   'asset': 107,
   'debt': 74,
   'equity': 33,
   'payable': 74}},
 '2001': {'TCB': {'cash': 29,
   'asset': 2388,
   'debt': 2273,
   'equity': 116,
   'payable': 2273},
  'BBC': {'cash': 13, 'asset': 163, 'debt': 80, 'equity': 83, 'payable': 80}},
 '2002': {'VNM': {'cash': 0, 'asset': 0, 'debt': 0, 'equity': 0, 'payable': 0},
  'TCB': {'cash': 63,
   'asset': 4060,
   'debt': 3929,
   'equity': 131,
   'payable': 3929},
  'STB': {'cash': 187,
   'asset': 4296,
   'debt': 3945,
   'equity': 352,
   'payable': 3945},
  'NHC': {'cash': 5, 'asset': 15, 'debt': 1, 'equity': 13, 'payable': 1},
  'FPT': {'cash': 103,
   'asset': 550,
   'debt': 517,
   'equity': 33,
   'payable': 517},
  'BBC': {'cash': 13, 'asset': 177, 'debt': 101, 'equity': 76, 'payable': 101},
  'AGF': {'cash': 2, 'asset': 168, 'debt': 102, 'equity': 66, 'payable': 102},
  'ACB': {'cash': 206,
   'asset': 9350,
   'debt': 8860,
   'equity': 489,
   'payable': 8860}},
 '2003': {'VSH': {'cas

In [5]:
# Select features for prediction
feature_cols = ['asset', 'debt', 'equity', 'cash', 'payable']
# feature_cols = ['asset']

In [6]:
# Convert balance_sheet dictionary to pandas DataFrame
rows = []
for year, companies in balance_sheet.items():
    for symbol, metrics in companies.items():
        row = {'year': year, 'symbol': symbol}
        row.update(metrics)
        rows.append(row)

df_balance_sheet = pd.DataFrame(rows)
df_balance_sheet = df_balance_sheet.sort_values(['symbol', 'year']).reset_index(drop=True)
df_balance_sheet = df_balance_sheet.fillna(value=0)
df_balance_sheet['year_int'] = df_balance_sheet['year'].str.split('-').str[0].astype(int)
if not yearly:
    df_balance_sheet['quarter'] = df_balance_sheet['year'].str.split('-Q').str[1].astype(int)
# Process each company separately
symbols = df_balance_sheet['symbol'].unique()

In [7]:
df_balance_sheet[df_balance_sheet['symbol'] == 'YEG']

,year,symbol,cash,asset,debt,equity,payable,year_int
22744,2012,YEG,11,86,40,45,40,2012
22745,2013,YEG,11,188,113,65,113,2013
22746,2014,YEG,12,245,154,91,154,2014
22747,2015,YEG,7,350,248,101,248,2015
22748,2016,YEG,13,355,245,110,245,2016
22749,2017,YEG,42,657,325,333,325,2017
22750,2018,YEG,192,1962,404,1558,404,2018
22751,2019,YEG,162,1515,494,1021,494,2019
22752,2020,YEG,36,1385,505,880,505,2020
22753,2021,YEG,4,1372,495,878,495,2021


In [8]:
import plotly.graph_objects as go
import plotly.express as px

# Get balance sheet data for correlation analysis
# Select multiple features for correlation
correlation_features = ['asset', 'debt', 'equity', 'cash', 'payable']

# Filter data to include only rows with complete feature data
df_corr = df_balance_sheet[correlation_features].copy()

# Calculate correlation matrix
correlation_matrix = df_corr.corr()

# Create interactive heatmap using Plotly
fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='RdBu',
    zmid=0,
    text=correlation_matrix.values,
    texttemplate='%{text:.2f}',
    textfont={"size": 12},
    colorbar=dict(title="Correlation")
))

fig.update_layout(
    title='Correlation Matrix - Balance Sheet Features',
    xaxis_title='Features',
    yaxis_title='Features',
    width=700,
    height=600,
    xaxis={'side': 'bottom'},
    yaxis={'autorange': 'reversed'}
)

fig.show()

In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Features to visualize
features_to_plot = ['asset', 'debt', 'equity', 'cash', 'payable']

# Select a few representative companies for better visualization
sample_symbols = df_balance_sheet['symbol'].unique()[:5]  # First 5 companies

# Create subplots for each feature
fig = make_subplots(
    rows=len(features_to_plot),
    cols=1,
    subplot_titles=[f'{feature.capitalize()} Trend Over Time' for feature in features_to_plot],
    vertical_spacing=0.08
)

# Add traces for each feature and company
for idx, feature in enumerate(features_to_plot, start=1):
    for symbol in sample_symbols:
        df_symbol = df_balance_sheet[df_balance_sheet['symbol'] == symbol].copy()
        df_symbol['year_int'] = df_symbol['year'].str.split('-').str[0].astype(int)
        df_symbol = df_symbol.sort_values('year_int')

        fig.add_trace(
            go.Scatter(
                x=df_symbol['year_int'],
                y=df_symbol[feature],
                mode='lines+markers',
                name=f'{symbol} - {feature}',
                legendgroup=symbol,
                showlegend=(idx == 1),  # Only show legend for first subplot
                hovertemplate=f'<b>{symbol}</b><br>Year: %{{x}}<br>{feature.capitalize()}: %{{y:,.0f}}B VND<extra></extra>'
            ),
            row=idx,
            col=1
        )

    # Update y-axis label
    fig.update_yaxes(title_text=f'{feature.capitalize()} (B VND)', row=idx, col=1)
    fig.update_xaxes(title_text='Year', row=idx, col=1)

# Update layout
fig.update_layout(
    height=300 * len(features_to_plot),
    title_text='Balance Sheet Features Trend Analysis',
    hovermode='x unified',
    showlegend=True,
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02
    )
)

fig.show()


# Task Summary
The user wants to explore balance sheet data to prepare it for feeding into an LSTM model. This involves data analysis, visualization, and understanding the data structure for machine learning.



# Balance Sheet Data Exploration for LSTM Model

This notebook explores balance sheet data to understand:
1. Data structure and completeness
2. Feature distributions and trends
3. Data quality issues (missing values, outliers)
4. Temporal patterns across companies
5. Feature correlations and relationships

## Key Findings Summary
- **Total Companies**: Analyzing balance sheet data for multiple companies
- **Time Range**: Historical data from 2000-2024
- **Key Features**: asset, debt, equity, cash, payable
- **Data Quality**: Check for missing values and outliers
- **Temporal Patterns**: Year-over-year growth trends

In [270]:
# Data Overview
print("=" * 80)
print("BALANCE SHEET DATA EXPLORATION")
print("=" * 80)

print(f"\n1. Dataset Shape: {df_balance_sheet.shape}")
print(f"   - Total Records: {len(df_balance_sheet):,}")
print(f"   - Features: {len(df_balance_sheet.columns)}")
print(f"   - Companies: {df_balance_sheet['symbol'].nunique()}")

print(f"\n2. Time Range:")
print(f"   - Earliest Year: {df_balance_sheet['year'].min()}")
print(f"   - Latest Year: {df_balance_sheet['year'].max()}")

print(f"\n3. Features Available:")
for col in df_balance_sheet.columns:
    if col not in ['year', 'symbol', 'year_int']:
        non_zero = (df_balance_sheet[col] != 0).sum()
        pct = (non_zero / len(df_balance_sheet)) * 100
        print(f"   - {col:15s}: {non_zero:6,} non-zero values ({pct:5.1f}%)")

print(f"\n4. Data Completeness by Company:")
records_per_symbol = df_balance_sheet.groupby('symbol').size()
print(f"   - Avg records per company: {records_per_symbol.mean():.1f}")
print(f"   - Min records: {records_per_symbol.min()}")
print(f"   - Max records: {records_per_symbol.max()}")

print(f"\n5. Missing Values:")
missing = df_balance_sheet[feature_cols].isnull().sum()
if missing.sum() == 0:
    print("   ✓ No missing values (filled with 0)")
else:
    print(missing)

print(f"\n6. Sample Companies for Detailed Analysis:")
sample_companies = df_balance_sheet['symbol'].unique()[:10]
for sym in sample_companies:
    company_data = df_balance_sheet[df_balance_sheet['symbol'] == sym]
    years = len(company_data)
    year_range = f"{company_data['year'].min()} to {company_data['year'].max()}"
    print(f"   - {sym:6s}: {years:2d} years ({year_range})")

print("\n" + "=" * 80)

BALANCE SHEET DATA EXPLORATION

1. Dataset Shape: (22780, 8)
   - Total Records: 22,780
   - Features: 8
   - Companies: 1583

2. Time Range:
   - Earliest Year: 2000
   - Latest Year: 2024

3. Features Available:
   - cash           : 21,918 non-zero values ( 96.2%)
   - asset          : 22,758 non-zero values ( 99.9%)
   - debt           : 22,711 non-zero values ( 99.7%)
   - equity         : 22,732 non-zero values ( 99.8%)
   - payable        : 22,711 non-zero values ( 99.7%)

4. Data Completeness by Company:
   - Avg records per company: 14.4
   - Min records: 1
   - Max records: 25

5. Missing Values:
   ✓ No missing values (filled with 0)

6. Sample Companies for Detailed Analysis:
   - A32   :  9 years (2016 to 2024)
   - AAA   : 18 years (2007 to 2024)
   - AAH   :  5 years (2020 to 2024)
   - AAM   : 20 years (2005 to 2024)
   - AAS   : 11 years (2013 to 2024)
   - AAT   : 11 years (2014 to 2024)
   - AAV   : 11 years (2014 to 2024)
   - ABB   : 19 years (2006 to 2024)
   - AB

In [271]:
# Statistical Summary by Feature
print("\nSTATISTICAL SUMMARY OF BALANCE SHEET FEATURES")
print("=" * 80)

for feature in ['asset', 'debt', 'equity', 'cash', 'payable']:
    print(f"\n{feature.upper()}:")
    stats = df_balance_sheet[feature].describe()
    print(f"  Mean:   {stats['mean']:15,.2f} Billion VND")
    print(f"  Median: {stats['50%']:15,.2f} Billion VND")
    print(f"  Std:    {stats['std']:15,.2f} Billion VND")
    print(f"  Min:    {stats['min']:15,.2f} Billion VND")
    print(f"  Max:    {stats['max']:15,.2f} Billion VND")

    # Calculate growth statistics
    df_growth = df_balance_sheet.copy()
    df_growth = df_growth.sort_values(['symbol', 'year_int'])
    df_growth[f'{feature}_growth'] = df_growth.groupby('symbol')[feature].pct_change() * 100

    avg_growth = df_growth[f'{feature}_growth'].mean()
    print(f"  Avg YoY Growth: {avg_growth:10,.2f}%")

print("\n" + "=" * 80)


STATISTICAL SUMMARY OF BALANCE SHEET FEATURES

ASSET:
  Mean:          7,777.64 Billion VND
  Median:          445.00 Billion VND
  Std:          67,563.78 Billion VND
  Min:               0.00 Billion VND
  Max:       2,760,792.00 Billion VND
  Avg YoY Growth:        inf%

DEBT:
  Mean:          6,338.41 Billion VND
  Median:          219.00 Billion VND
  Std:          62,233.80 Billion VND
  Min:               0.00 Billion VND
  Max:       2,615,881.00 Billion VND
  Avg YoY Growth:        inf%

EQUITY:
  Mean:          1,429.63 Billion VND
  Median:          184.00 Billion VND
  Std:           7,023.67 Billion VND
  Min:         -17,026.00 Billion VND
  Max:         220,744.00 Billion VND
  Avg YoY Growth:        nan%

CASH:
  Mean:            238.47 Billion VND
  Median:           23.00 Billion VND
  Std:           1,149.69 Billion VND
  Min:               0.00 Billion VND
  Max:          42,582.00 Billion VND
  Avg YoY Growth:        inf%

PAYABLE:
  Mean:          6,338.41 Billio

/Users/tuyenhuynh/Documents/Study/FinalProject/FinancialAssistant/notebook/.venv/lib/python3.11/site-packages/numpy/_core/_methods.py:51: RuntimeWarning:

invalid value encountered in reduce



In [272]:
look_back = 3
# Identify companies with most complete data
print("\nCOMPANIES WITH MOST COMPLETE HISTORICAL DATA")
print("=" * 80)

company_completeness = df_balance_sheet.groupby('symbol').agg({
    'year_int': ['count', 'min', 'max'],
    'asset': 'mean'
}).round(2)

company_completeness.columns = ['num_years', 'first_year', 'last_year', 'avg_asset']
company_completeness = company_completeness.sort_values('num_years', ascending=False)

print("\nTop 20 Companies by Data Completeness:")
print(company_completeness.head(20).to_string())

print(f"\n\nCompanies with >= {look_back + 1} years of data (suitable for LSTM):")
suitable_companies = company_completeness[company_completeness['num_years'] >= (look_back + 1)]
print(f"Count: {len(suitable_companies)} companies")

print("\n" + "=" * 80)


COMPANIES WITH MOST COMPLETE HISTORICAL DATA

Top 20 Companies by Data Completeness:
        num_years  first_year  last_year  avg_asset
symbol                                             
BBC            25        2000       2024     898.72
TCB            24        2001       2024  250894.92
ACB            23        2002       2024  263784.48
FPT            23        2002       2024   23291.22
AGF            23        2002       2024    1064.74
STB            23        2002       2024  259808.04
NHC            23        2002       2024      49.17
VNM            23        2002       2024   24296.48
CAN            22        2003       2024     230.73
ALT            22        2003       2024     236.91
BID            22        2003       2024  909021.91
REE            22        2003       2024   12381.27
MBB            22        2003       2024  293167.23
LAF            22        2003       2024     280.05
SAM            22        2003       2024    3678.64
GIL            22        2003 

In [273]:
# Analyze data distribution and outliers
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("\nDATA DISTRIBUTION ANALYSIS")
print("=" * 80)

# Create distribution plots
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=['Asset Distribution', 'Debt Distribution', 'Equity Distribution',
                    'Cash Distribution', 'Payable Distribution', 'Asset Log Scale'],
    specs=[[{"type": "histogram"}, {"type": "histogram"}, {"type": "histogram"}],
           [{"type": "histogram"}, {"type": "histogram"}, {"type": "histogram"}]]
)

features = ['asset', 'debt', 'equity', 'cash', 'payable']
positions = [(1, 1), (1, 2), (1, 3), (2, 1), (2, 2)]

for feature, (row, col) in zip(features, positions):
    fig.add_trace(
        go.Histogram(x=df_balance_sheet[feature], name=feature, nbinsx=50),
        row=row, col=col
    )

# Log scale for asset
fig.add_trace(
    go.Histogram(x=np.log10(df_balance_sheet['asset'] + 1), name='log(asset)', nbinsx=50),
    row=2, col=3
)

fig.update_layout(
    height=800,
    title_text="Balance Sheet Features Distribution",
    showlegend=False
)

fig.show()

# Print outlier statistics
print("\nOutlier Detection (values > 99th percentile):")
for feature in features:
    p99 = df_balance_sheet[feature].quantile(0.99)
    outliers = df_balance_sheet[df_balance_sheet[feature] > p99]
    print(f"\n{feature.upper()}:")
    print(f"  99th percentile: {p99:,.2f} Billion VND")
    print(f"  Outliers count: {len(outliers)}")
    if len(outliers) > 0:
        print(f"  Top outlier companies: {', '.join(outliers.nlargest(5, feature)['symbol'].values)}")

print("\n" + "=" * 80)



DATA DISTRIBUTION ANALYSIS



Outlier Detection (values > 99th percentile):

ASSET:
  99th percentile: 150,291.63 Billion VND
  Outliers count: 228
  Top outlier companies: BID, CTG, BID, BID, VCB

DEBT:
  99th percentile: 130,131.97 Billion VND
  Outliers count: 228
  Top outlier companies: BID, CTG, BID, BID, CTG

EQUITY:
  99th percentile: 25,936.44 Billion VND
  Outliers count: 228
  Top outlier companies: VHM, VCB, VHM, VCB, VIC

CASH:
  99th percentile: 4,934.56 Billion VND
  Outliers count: 228
  Top outlier companies: VIC, VIC, BSR, VHM, VIC

PAYABLE:
  99th percentile: 130,131.97 Billion VND
  Outliers count: 228
  Top outlier companies: BID, CTG, BID, BID, CTG



In [274]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, GRU
from tensorflow.keras.callbacks import EarlyStopping
import datetime

In [292]:
# Create sequences for LSTM
def create_sequences(dataset, look_back_years):
    """
    Create input-output sequences for LSTM model training using sliding window approach.
    
    This function transforms a time series dataset into sequences that can be used for
    supervised learning with LSTM networks. For each position in the dataset, it creates
    a sequence of 'look_back_years' consecutive data points as input (X) and the next
    data point as the target (y).
    
    Parameters:
    -----------
    dataset : numpy.ndarray
        A 2D array where each row represents a time step and each column represents a feature.
        Shape: (n_samples, n_features)
        Example: For balance sheet data, each row could be [asset, debt, equity] for a year.
    
    look_back_years : int
        The number of previous time steps to use as input features (window size).
        This determines how much historical context the LSTM will consider.
        Example: If look_back_years=5, the model will use 5 consecutive years to predict the 6th.
    
    Returns:
    --------
    X : numpy.ndarray
        Input sequences for the LSTM model.
        Shape: (n_sequences, look_back_years, n_features)
        Each element X[i] contains look_back_years consecutive rows from the dataset.
    
    y : numpy.ndarray
        Target values corresponding to each input sequence.
        Shape: (n_sequences, n_features)
        Each element y[i] is the data point immediately following the sequence in X[i].
    
    How it works:
    ------------
    The function uses a sliding window technique:
    
    1. For a dataset with N rows and look_back_years = L:
       - It creates (N - L) sequences
       - Each sequence uses L consecutive rows as input
       - The target is the (L+1)th row
    
    2. Example with dataset = [[1], [2], [3], [4], [5], [6]] and look_back_years = 3:
       - X[0] = [[1], [2], [3]], y[0] = [4]
       - X[1] = [[2], [3], [4]], y[1] = [5]
       - X[2] = [[3], [4], [5]], y[2] = [6]
    
    3. For multi-feature data (e.g., [asset, debt, equity]):
       - Each X[i] contains look_back_years rows, each with all features
       - Each y[i] contains the next row with all features
    
    Notes:
    ------
    - The function assumes the dataset is already preprocessed (scaled, normalized, etc.)
    - The dataset should be sorted chronologically for time series prediction
    - Missing data should be handled before calling this function
    """
    # Initialize empty lists to store input sequences (X) and target values (y)
    X, y = [], []
    
    # Slide a window of size 'look_back_years' through the dataset
    # We can create sequences from index 0 to (len(dataset) - look_back_years - 1)
    for i in range(len(dataset) - look_back_years):
        # Extract input sequence: 'look_back_years' consecutive rows starting from index i
        # This represents historical data points used to predict the next value
        X.append(dataset[i:(i + look_back_years)])

        # Extract target value: the row immediately following the input sequence
        # This is what we want the LSTM model to predict
        y.append(dataset[i + look_back_years])

    # Convert lists to numpy arrays for efficient computation and LSTM compatibility
    # X shape: (n_sequences, look_back_years, n_features)
    # y shape: (n_sequences, n_features)
    return np.array(X), np.array(y)

In [293]:
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import LeakyReLU

# Configuration
today = datetime.datetime.today()
training_end_year = 2022
look_back = today.year - training_end_year
feature_cols = ['asset']
# Prepare data
df_train = df_balance_sheet.copy()
df_train['year_int'] = df_train['year'].str.split('-').str[0].astype("int")

# Prepare training and validation data for all symbols
train_X, train_y = [], []
val_X, val_y = [], []

for symbol in df_train['symbol'].unique():
    if symbol != 'VNM':
        continue
    df_symbol = df_train[df_train['symbol'] == symbol].copy()

    # Split: training data up to 2022, validation data from 2023 onwards
    train_mask = df_symbol['year_int'] <= training_end_year
    val_mask = df_symbol['year_int'] >= training_end_year + 1

    df_train_symbol = df_symbol[train_mask].sort_values('year_int')
    df_val_symbol = df_symbol[val_mask].sort_values('year_int')

    # Skip if insufficient training data
    if len(df_train_symbol) < look_back + 1:
        print(f"Insufficient data for {symbol}. Need at least {look_back + 1} years.")
        continue

    # Extract features and scale (fit scaler on training data only)
    train_data = df_train_symbol[feature_cols].values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_train_data = scaler.fit_transform(train_data)

    # Create training sequences
    X, y = create_sequences(scaled_train_data, look_back)

    if len(X) > 0:
        train_X.append(X)
        train_y.append(y)

    # Create validation sequences if validation data exists
    if len(df_val_symbol) > 0:
        # Use last look_back points from training + validation data
        combined_data = np.vstack([train_data[-look_back:], df_val_symbol[feature_cols].values])
        scaled_combined = scaler.transform(combined_data)

        X_val, y_val = create_sequences(scaled_combined, look_back)

        if len(X_val) > 0:
            val_X.append(X_val)
            val_y.append(y_val)

# Combine all sequences
X_train = np.vstack(train_X)
y_train = np.vstack(train_y)

print(f"Training data shape: X={X_train.shape}, y={y_train.shape}")
print(f"Training period: historical data to {training_end_year}")
print(f"Look-back window: {look_back} years")

# Prepare validation data if available
if len(val_X) > 0:
    X_val = np.vstack(val_X)
    y_val = np.vstack(val_y)
    print(f"Validation data shape: X={X_val.shape}, y={y_val.shape}")
    print(f"Validation period: 2023 to {today.year}")
    validation_data = (X_val, y_val)
else:
    print("No validation data available")
    validation_data = None

# Build LSTM model
model = Sequential([
    Input(shape=(look_back, len(feature_cols))),
    LSTM(50, activation='relu', return_sequences=True),
    # LeakyReLU(),
    GRU(50, activation='relu', return_sequences=True),
    Dropout(0.3),
    LSTM(50, activation='relu'),
    # LeakyReLU(),
    Dropout(0.3),
    Dense(len(feature_cols))
])

model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

# Train model with validation data
early_stopping = EarlyStopping(
    monitor='val_loss' if validation_data else 'loss',
    patience=20,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_data=validation_data,
    epochs=100,
    batch_size=32,
    verbose=1,
    callbacks=[early_stopping]
)

# Randomly shuffle data for train-validation split
indices = np.arange(len(X_train))
np.random.seed(42)
np.random.shuffle(indices)

split_idx = int(len(X_train) * (0.8))
train_indices = indices[:split_idx]
val_indices = indices[split_idx:]

X_train_split = X_train[train_indices]
y_train_split = y_train[train_indices]
X_val_split = X_train[val_indices]
y_val_split = y_train[val_indices]

# Evaluate on training and validation sets
train_loss = model.evaluate(X_train_split, y_train_split, verbose=0)
val_loss = model.evaluate(X_val_split, y_val_split, verbose=0)

# Save model
model.save('balancesheet_lstm_model_all_features.keras')
print("\n" + "=" * 80)
print("Model training completed and saved!")
print(f"Training samples: {len(X_train):,}")
if validation_data:
    print(f"Validation samples: {len(X_val):,}")
print("=" * 80)

print(f"\nModel Performance:")
print(f"  Training Loss (MSE): {train_loss:.6f}")
print(f"  Validation Loss (MSE): {val_loss:.6f}")

# Make predictions for visualization
y_train_pred = model.predict(X_train_split, verbose=0)
y_val_pred = model.predict(X_val_split, verbose=0)

# Calculate additional metrics
train_mse = mean_squared_error(y_train_split, y_train_pred)
val_mse = mean_squared_error(y_val_split, y_val_pred)

print(f"\nDetailed Metrics:")
print(f"  Training MSE: {train_mse:.6f}")
print(f"  Validation MSE: {val_mse:.6f}")

Training data shape: X=(18, 3, 1), y=(18, 1)
Training period: historical data to 2022
Look-back window: 3 years
Validation data shape: X=(2, 3, 1), y=(2, 1)
Validation period: 2023 to 2025


Model: "sequential_34"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_66 (LSTM)                  │ (None, 3, 50)          │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_33 (GRU)                    │ (None, 3, 50)          │        15,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_66 (Dropout)            │ (None, 3, 50)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_67 (LSTM)                  │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_67 (Dropout)            │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 45,951 (179.50 KB)

 Trainable params: 45,951 (179.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.3076 - val_loss: 1.0020
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.3034 - val_loss: 0.9887
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.2977 - val_loss: 0.9760
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.2919 - val_loss: 0.9630
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.2859 - val_loss: 0.9495
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.2825 - val_loss: 0.9356
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.2746 - val_loss: 0.9211
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.2726 - val_loss: 0.9060
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.2634 - val_loss: 0.8900
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.2552 - val_loss: 0.8733
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.2552 - val_loss: 0.8556
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.2466 - val_loss

In [294]:
# Predict asset of VNM symbol for user-specified year and visualize
from tensorflow.keras.models import load_model
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import datetime

# Load the saved model
loaded_model = load_model('balancesheet_lstm_model_all_features.keras')

# Configuration
symbol_prediction = "VNM"
# symbol_prediction = str(input("Enter the symbol you want to predict for 2023: ")).upper()
today = datetime.datetime.today()
look_back = 3
# FIXED: Use the same features as during training
# feature_cols_prediction = ['asset', 'debt', 'equity', 'cash', 'payable']
feature_cols_prediction = ['asset']

# User input for prediction year
prediction_year = 2023
future_years = [2023]

# Get symbol data
df_symbol = df_balance_sheet[df_balance_sheet['symbol'] == symbol_prediction].copy()
df_symbol['year_int'] = df_symbol['year'].str.split('-').str[0].astype(int)

# Check if we have enough data
if len(df_symbol) < look_back + 2:
    print(f"Insufficient data for {symbol_prediction}. Need at least {look_back + 2} years.")
else:
    # Prepare data - Use ALL historical data for fitting the scaler
    data = df_symbol[feature_cols_prediction].values

    # Normalize data using ALL data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    # Get data up to year before prediction for training
    df_before_prediction = df_symbol[df_symbol['year_int'] < prediction_year].copy().sort_values("year_int", ascending=True)
    data_before_prediction = df_before_prediction[feature_cols_prediction].values

    # Transform (not fit_transform) using the same scaler
    scaled_data_before_prediction = scaler.transform(data_before_prediction)

    # Predict for the specified year(s)
    last_sequence = scaled_data_before_prediction[-look_back:].copy()
    future_predictions = []

    # Calculate number of years to predict
    num_years_to_predict = prediction_year - df_before_prediction['year_int'].max()
    print(f"Number of years to predict: {num_years_to_predict}")

    for _ in range(num_years_to_predict):
        pred_input = last_sequence.reshape(1, look_back, len(feature_cols_prediction))
        pred = loaded_model.predict(pred_input, verbose=0)
        future_predictions.append(pred[0])
        last_sequence = np.vstack([last_sequence[1:], pred[0]])

    # Inverse transform predictions
    future_predictions = scaler.inverse_transform(np.array(future_predictions))

    # Prepare actual data
    actual_data = scaler.inverse_transform(scaled_data)

    # Calculate metrics if actual data is available for prediction year
    actual_prediction_mask = df_symbol['year_int'] == prediction_year
    predicted_value = future_predictions[-1, 0]  # Get the last prediction (target year)

    # Initialize variables for annotation
    annotation_text = ""
    actual_value = None

    if actual_prediction_mask.any():
        actual_values = actual_data[actual_prediction_mask.values].flatten()
        predicted_values = future_predictions[-1]

        mse = mean_squared_error(actual_values, predicted_values)
        rmse = np.sqrt(mse)

        # Calculate training data statistics
        train_data = df_before_prediction[feature_cols_prediction].values
        avg_growth = np.mean(np.diff(train_data[-5:, 0]) / train_data[-5:-1, 0] * 100)

        # Get comparison values
        actual_value = actual_data[actual_prediction_mask.values, 0][0]
        difference = predicted_value - actual_value
        percentage_diff = (difference / actual_value) * 100 if actual_value != 0 else 0

        # Build annotation text
        annotation_text = f"<b>{prediction_year} Prediction for {symbol_prediction}</b><br>"
        annotation_text += f"<b>Asset</b>: {predicted_value:,.2f} Billion VND<br><br>"

        annotation_text += f"<b>Comparison with Actual {prediction_year}:</b><br>"
        annotation_text += f"• Actual {prediction_year}: {actual_value:,.2f} Billion VND<br>"
        annotation_text += f"• Difference: {difference:,.2f} ({percentage_diff:+.2f}%)<br><br>"

        annotation_text += f"<b>Prediction Metrics:</b><br>"
        annotation_text += f"• MSE: {mse:,.2f}<br>"
        annotation_text += f"• RMSE: {rmse:,.2f}<br><br>"

        annotation_text += f"<b>Training Data Statistics:</b><br>"
        annotation_text += f"• Avg Growth Rate (last 5 years): {avg_growth:.2f}%<br>"
        # annotation_text += f"• Training Range: {df_before_prediction['year_int'].min()}-{df_before_prediction['year_int'].max()}<br><br>"
    else:
        annotation_text = f"<b>{prediction_year} Prediction for {symbol_prediction}</b><br>"
        annotation_text += f"<b>Asset</b>: {predicted_value:,.2f} Billion VND<br><br>"
        annotation_text += f"<b>Training Data Statistics:</b><br>"
        train_data = df_before_prediction[feature_cols_prediction].values
        avg_growth = np.mean(np.diff(train_data[-5:, 0]) / train_data[-5:-1, 0] * 100)
        annotation_text += f"• Avg Growth Rate (last 5 years): {avg_growth:.2f}%<br>"
        # annotation_text += f"• Training Range: {df_before_prediction['year_int'].min()}-{df_before_prediction['year_int'].max()}<br>"

    # Create interactive Plotly chart
    fig = go.Figure()

    # Add historical data trace
    fig.add_trace(
        go.Scatter(
            x=df_symbol['year_int'],
            y=actual_data[:, 0],
            mode='lines+markers',
            name='Historical',
            line=dict(color='blue', width=2),
            marker=dict(size=8),
            hovertemplate='Year: %{x}<br>Asset: %{y:,.2f}B VND<extra></extra>'
        )
    )

    # fig.add_trace(
    #     go.Scatter(
    #         x=df_before_prediction['year_int'],
    #         y=actual_data[:len(df_before_prediction), 0],
    #         mode='lines+markers',
    #         name='Training Data',
    #         line=dict(color='purple', width=2),
    #         marker=dict(size=8),
    #         hovertemplate='Year: %{x}<br>Asset: %{y:,.2f}B VND<extra></extra>'
    #     )
    # )

    # Add prediction trace
    fig.add_trace(
        go.Scatter(
            x=future_years,
            y=[future_predictions[-1, 0]],
            mode='lines+markers',
            name='Predicted',
            line=dict(color='red', width=2, dash='dash'),
            marker=dict(color='red', size=12, symbol='square'),
            hovertemplate='Year: %{x}<br>Predicted: %{y:,.2f}B VND<extra></extra>'
        )
    )

    # Add actual point if available
    if actual_prediction_mask.any():
        fig.add_trace(
            go.Scatter(
                x=[prediction_year],
                y=[actual_value],
                mode='markers',
                name=f'Actual {prediction_year}',
                marker=dict(color='green', size=14, symbol='diamond'),
                hovertemplate='Year: %{x}<br>Actual: %{y:,.2f}B VND<extra></extra>'
            )
        )

    # Update layout
    fig.update_layout(
        title=f'{symbol_prediction} - Asset LSTM Prediction for {prediction_year}',
        xaxis_title='Year',
        yaxis_title='Asset (Billion VND)',
        height=600,
        showlegend=True,
        hovermode='x unified',
        template='plotly_white'
    )

    # Add annotation
    if annotation_text:
        fig.add_annotation(
            text=annotation_text,
            xref="paper", yref="paper",
            x=0.02, y=0.98,
            xanchor='left', yanchor='top',
            showarrow=False,
            bgcolor="rgba(255, 255, 255, 0.9)",
            bordercolor="black",
            borderwidth=1,
            font=dict(size=10),
            align='left'
        )

    fig.show()


Number of years to predict: 1


In [263]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
import plotly.graph_objects as go

# Configuration
symbol_prediction = str(input("Enter the symbol you want to predict for 2023: "))
prediction_year = 2023
look_back = 3
feature_cols_prediction = ['asset']

# Get symbol data
df_symbol = df_balance_sheet[df_balance_sheet['symbol'] == symbol_prediction].copy()
df_symbol['year_int'] = df_symbol['year'].str.split('-').str[0].astype(int)

# Check if we have enough data
if len(df_symbol) < look_back + 2:
    print(f"Insufficient data for {symbol_prediction}. Need at least {look_back + 2} years.")
else:
    # Prepare data
    df_before_prediction = df_symbol[df_symbol['year_int'] < prediction_year].copy()

    # Create sequences for Linear Regression
    X_train = []
    y_train = []

    data = df_before_prediction[feature_cols_prediction].values

    for i in range(len(data) - look_back):
        X_train.append(data[i:(i + look_back)].flatten())
        y_train.append(data[i + look_back])

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    # Train Linear Regression model
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)

    # Make prediction for 2023
    last_sequence = data[-look_back:].flatten().reshape(1, -1)
    predicted_value = lr_model.predict(last_sequence)[0][0]

    # Get actual value if available
    actual_prediction_mask = df_symbol['year_int'] == prediction_year
    actual_value = None

    # Build annotation text
    annotation_text = f"<b>{prediction_year} Linear Regression Prediction for {symbol_prediction}</b><br>"
    annotation_text += f"<b>Asset</b>: {predicted_value:,.2f} Billion VND<br><br>"

    if actual_prediction_mask.any():
        actual_value = df_symbol[actual_prediction_mask][feature_cols_prediction[0]].values[0]
        difference = predicted_value - actual_value
        percentage_diff = (difference / actual_value) * 100 if actual_value != 0 else 0

        # Calculate model metrics on training data
        y_train_pred = lr_model.predict(X_train)
        train_r2 = r2_score(y_train, y_train_pred)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

        annotation_text += f"<b>Comparison with Actual {prediction_year}:</b><br>"
        annotation_text += f"• Actual {prediction_year}: {actual_value:,.2f} Billion VND<br>"
        annotation_text += f"• Difference: {difference:,.2f} ({percentage_diff:+.2f}%)<br><br>"

        annotation_text += f"<b>Model Performance:</b><br>"
        annotation_text += f"• R² Score: {train_r2:.4f}<br>"
        annotation_text += f"• RMSE: {train_rmse:,.2f}<br><br>"

    annotation_text += f"<b>Training Data:</b><br>"
    annotation_text += f"• Training Range: {df_before_prediction['year_int'].min()}-{df_before_prediction['year_int'].max()}<br>"
    annotation_text += f"• Look-back Window: {look_back} years<br>"
    annotation_text += f"• Training Samples: {len(X_train)}<br>"

    # Create visualization
    fig = go.Figure()

    # Historical data
    fig.add_trace(
        go.Scatter(
            x=df_symbol['year_int'],
            y=df_symbol['asset'],
            mode='lines+markers',
            name='Historical',
            line=dict(color='blue', width=2),
            marker=dict(size=8),
            hovertemplate='Year: %{x}<br>Asset: %{y:,.2f}B VND<extra></extra>'
        )
    )

    # Training data
    fig.add_trace(
        go.Scatter(
            x=df_before_prediction['year_int'],
            y=df_before_prediction['asset'],
            mode='lines+markers',
            name='Training Data',
            line=dict(color='purple', width=2),
            marker=dict(size=8),
            hovertemplate='Year: %{x}<br>Asset: %{y:,.2f}B VND<extra></extra>'
        )
    )

    # Prediction
    fig.add_trace(
        go.Scatter(
            x=[prediction_year],
            y=[predicted_value],
            mode='markers',
            name='Linear Regression Prediction',
            marker=dict(color='red', size=12, symbol='square'),
            hovertemplate='Year: %{x}<br>Predicted: %{y:,.2f}B VND<extra></extra>'
        )
    )

    # Actual value if available
    if actual_prediction_mask.any():
        fig.add_trace(
            go.Scatter(
                x=[prediction_year],
                y=[actual_value],
                mode='markers',
                name=f'Actual {prediction_year}',
                marker=dict(color='green', size=14, symbol='diamond'),
                hovertemplate='Year: %{x}<br>Actual: %{y:,.2f}B VND<extra></extra>'
            )
        )

    # Update layout
    fig.update_layout(
        title=f'{symbol_prediction} - Asset Linear Regression Prediction for {prediction_year}',
        xaxis_title='Year',
        yaxis_title='Asset (Billion VND)',
        height=600,
        showlegend=True,
        hovermode='x unified',
        template='plotly_white'
    )

    # Add annotation
    fig.add_annotation(
        text=annotation_text,
        xref="paper", yref="paper",
        x=0.02, y=0.98,
        xanchor='left', yanchor='top',
        showarrow=False,
        bgcolor="rgba(255, 255, 255, 0.9)",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=10),
        align='left'
    )

    fig.show()


Insufficient data for vnm. Need at least 5 years.


In [295]:
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import LeakyReLU

# Configuration
today = datetime.datetime.today()
training_end_year = 2022
look_back = today.year - training_end_year
feature_cols = ['asset']
# Prepare data
df_train = df_balance_sheet.copy()
df_train['year_int'] = df_train['year'].str.split('-').str[0].astype("int")

# Prepare training and validation data for all symbols
train_X, train_y = [], []
val_X, val_y = [], []

for symbol in df_train['symbol'].unique():
    df_symbol = df_train[df_train['symbol'] == symbol].copy()

    # Split: training data up to 2022, validation data from 2023 onwards
    train_mask = df_symbol['year_int'] <= training_end_year
    val_mask = df_symbol['year_int'] >= training_end_year + 1

    df_train_symbol = df_symbol[train_mask].sort_values('year_int')
    df_val_symbol = df_symbol[val_mask].sort_values('year_int')

    # Skip if insufficient training data
    if len(df_train_symbol) < look_back + 1:
        print(f"Insufficient data for {symbol}. Need at least {look_back + 1} years.")
        continue

    # Extract features and scale (fit scaler on training data only)
    train_data = df_train_symbol[feature_cols].values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_train_data = scaler.fit_transform(train_data)

    # Create training sequences
    X, y = create_sequences(scaled_train_data, look_back)

    if len(X) > 0:
        train_X.append(X)
        train_y.append(y)

    # Create validation sequences if validation data exists
    if len(df_val_symbol) > 0:
        # Use last look_back points from training + validation data
        combined_data = np.vstack([train_data[-look_back:], df_val_symbol[feature_cols].values])
        scaled_combined = scaler.transform(combined_data)

        X_val, y_val = create_sequences(scaled_combined, look_back)

        if len(X_val) > 0:
            val_X.append(X_val)
            val_y.append(y_val)

# Combine all sequences
X_train = np.vstack(train_X)
y_train = np.vstack(train_y)

print(f"Training data shape: X={X_train.shape}, y={y_train.shape}")
print(f"Training period: historical data to {training_end_year}")
print(f"Look-back window: {look_back} years")

# Prepare validation data if available
if len(val_X) > 0:
    X_val = np.vstack(val_X)
    y_val = np.vstack(val_y)
    print(f"Validation data shape: X={X_val.shape}, y={y_val.shape}")
    print(f"Validation period: 2023 to {today.year}")
    validation_data = (X_val, y_val)
else:
    print("No validation data available")
    validation_data = None

# Build LSTM model
model = Sequential([
    Input(shape=(look_back, len(feature_cols))),
    LSTM(50, activation='relu', return_sequences=True),
    # LeakyReLU(),
    GRU(50, activation='relu', return_sequences=True),
    Dropout(0.3),
    LSTM(50, activation='relu'),
    # LeakyReLU(),
    Dropout(0.3),
    Dense(len(feature_cols))
])

model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

# Train model with validation data
early_stopping = EarlyStopping(
    monitor='val_loss' if validation_data else 'loss',
    patience=20,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_data=validation_data,
    epochs=100,
    batch_size=32,
    verbose=1,
    callbacks=[early_stopping]
)

# Randomly shuffle data for train-validation split
indices = np.arange(len(X_train))
np.random.seed(42)
np.random.shuffle(indices)

split_idx = int(len(X_train) * (0.8))
train_indices = indices[:split_idx]
val_indices = indices[split_idx:]

X_train_split = X_train[train_indices]
y_train_split = y_train[train_indices]
X_val_split = X_train[val_indices]
y_val_split = y_train[val_indices]

# Evaluate on training and validation sets
train_loss = model.evaluate(X_train_split, y_train_split, verbose=0)
val_loss = model.evaluate(X_val_split, y_val_split, verbose=0)

# Save model
model.save('balancesheet_lstm_model_all_features_all_symbols.keras')
print("\n" + "=" * 80)
print("Model training completed and saved!")
print(f"Training samples: {len(X_train):,}")
if validation_data:
    print(f"Validation samples: {len(X_val):,}")
print("=" * 80)

print(f"\nModel Performance:")
print(f"  Training Loss (MSE): {train_loss:.6f}")
print(f"  Validation Loss (MSE): {val_loss:.6f}")

# Make predictions for visualization
y_train_pred = model.predict(X_train_split, verbose=0)
y_val_pred = model.predict(X_val_split, verbose=0)

# Calculate additional metrics
train_mse = mean_squared_error(y_train_split, y_train_pred)
val_mse = mean_squared_error(y_val_split, y_val_pred)

print(f"\nDetailed Metrics:")
print(f"  Training MSE: {train_mse:.6f}")
print(f"  Validation MSE: {val_mse:.6f}")

Insufficient data for AAH. Need at least 4 years.
Insufficient data for ACG. Need at least 4 years.
Insufficient data for AIC. Need at least 4 years.
Insufficient data for AIG. Need at least 4 years.
Insufficient data for AVG. Need at least 4 years.
Insufficient data for BAF. Need at least 4 years.
Insufficient data for BCA. Need at least 4 years.
Insufficient data for BCR. Need at least 4 years.
Insufficient data for BGE. Need at least 4 years.
Insufficient data for BHI. Need at least 4 years.
Insufficient data for BIG. Need at least 4 years.
Insufficient data for BNA. Need at least 4 years.
Insufficient data for CAR. Need at least 4 years.
Insufficient data for CCC. Need at least 4 years.
Insufficient data for CMM. Need at least 4 years.
Insufficient data for CNA. Need at least 4 years.
Insufficient data for CRV. Need at least 4 years.
Insufficient data for D17. Need at least 4 years.
Insufficient data for DAN. Need at least 4 years.
Insufficient data for DDB. Need at least 4 years.


Model: "sequential_35"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_68 (LSTM)                  │ (None, 3, 50)          │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_34 (GRU)                    │ (None, 3, 50)          │        15,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_68 (Dropout)            │ (None, 3, 50)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_69 (LSTM)                  │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_69 (Dropout)            │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 45,951 (179.50 KB)

 Trainable params: 45,951 (179.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
473/473 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0746 - val_loss: 0.2370
Epoch 2/100
473/473 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0431 - val_loss: 0.2431
Epoch 3/100
473/473 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0404 - val_loss: 0.2519
Epoch 4/100
473/473 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0395 - val_loss: 0.2618
Epoch 5/100
473/473 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0383 - val_loss: 0.2704
Epoch 6/100
473/473 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0372 - val_loss: 0.2725
Epoch 7/100
473/473 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0369 - val_loss: 0.2812
Epoch 8/100
473/473 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0367 - val_loss: 0.2925
Epoch 9/100
473/473 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0360 - val_loss: 0.2943
Epoch 10/100
473/473 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0357 - val_loss: 0.3029
Epoch 11/100
473/473 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0356 - val_loss: 0.3027
Epoch 12/100
473/473 ━━━━━━━━━━━━━━━━━━━━

In [1]:
# Predict asset of VNM symbol for user-specified year and visualize
from tensorflow.keras.models import load_model
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import datetime

# Load the saved model
loaded_model = load_model('balancesheet_lstm_model_all_features_all_symbols.keras')

# Configuration
# symbol_prediction = "VNM"
symbol_prediction = str(input("Enter the symbol you want to predict for 2023: ")).upper()
today = datetime.datetime.today()
look_back = 6
# FIXED: Use the same features as during training
# feature_cols_prediction = ['asset', 'debt', 'equity', 'cash', 'payable']
feature_cols_prediction = ['asset']

# User input for prediction year
prediction_year = 2024
future_years = [2023, 2024]

# Get symbol data
df_symbol = df_balance_sheet[df_balance_sheet['symbol'] == symbol_prediction].copy()
df_symbol['year_int'] = df_symbol['year'].str.split('-').str[0].astype(int)

# Check if we have enough data
if len(df_symbol) < look_back + 2:
    print(f"Insufficient data for {symbol_prediction}. Need at least {look_back + 2} years.")
else:
    # Prepare data - Use ALL historical data for fitting the scaler
    data = df_symbol[feature_cols_prediction].values

    # Normalize data using ALL data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    # Get data up to year before prediction for training
    df_before_prediction = df_symbol[df_symbol['year_int'] < prediction_year].copy().sort_values("year_int", ascending=True)
    data_before_prediction = df_before_prediction[feature_cols_prediction].values

    # Transform (not fit_transform) using the same scaler
    scaled_data_before_prediction = scaler.transform(data_before_prediction)

    # Predict for the specified year(s)
    last_sequence = scaled_data_before_prediction[-look_back:].copy()
    future_predictions = []

    # Calculate number of years to predict
    num_years_to_predict = prediction_year - df_before_prediction['year_int'].max()
    print(f"Number of years to predict: {num_years_to_predict}")

    for _ in range(num_years_to_predict):
        pred_input = last_sequence.reshape(1, look_back, len(feature_cols_prediction))
        pred = loaded_model.predict(pred_input, verbose=0)
        future_predictions.append(pred[0])
        last_sequence = np.vstack([last_sequence[1:], pred[0]])

    # Inverse transform predictions
    future_predictions = scaler.inverse_transform(np.array(future_predictions))

    # Prepare actual data
    actual_data = scaler.inverse_transform(scaled_data)

    # Calculate metrics if actual data is available for prediction year
    actual_prediction_mask = df_symbol['year_int'] == prediction_year
    predicted_value = future_predictions[-1, 0]  # Get the last prediction (target year)

    # Initialize variables for annotation
    annotation_text = ""
    actual_value = None

    if actual_prediction_mask.any():
        actual_values = actual_data[actual_prediction_mask.values].flatten()
        predicted_values = future_predictions[-1]

        mse = mean_squared_error(actual_values, predicted_values)
        rmse = np.sqrt(mse)

        # Calculate training data statistics
        train_data = df_before_prediction[feature_cols_prediction].values
        avg_growth = np.mean(np.diff(train_data[-5:, 0]) / train_data[-5:-1, 0] * 100)

        # Get comparison values
        actual_value = actual_data[actual_prediction_mask.values, 0][0]
        difference = predicted_value - actual_value
        percentage_diff = (difference / actual_value) * 100 if actual_value != 0 else 0

        # Build annotation text
        annotation_text = f"<b>{prediction_year} Prediction for {symbol_prediction}</b><br>"
        annotation_text += f"<b>Asset</b>: {predicted_value:,.2f} Billion VND<br><br>"

        annotation_text += f"<b>Comparison with Actual {prediction_year}:</b><br>"
        annotation_text += f"• Actual {prediction_year}: {actual_value:,.2f} Billion VND<br>"
        annotation_text += f"• Difference: {difference:,.2f} ({percentage_diff:+.2f}%)<br><br>"

        annotation_text += f"<b>Prediction Metrics:</b><br>"
        annotation_text += f"• MSE: {mse:,.2f}<br>"
        annotation_text += f"• RMSE: {rmse:,.2f}<br><br>"

        annotation_text += f"<b>Training Data Statistics:</b><br>"
        annotation_text += f"• Avg Growth Rate (last 5 years): {avg_growth:.2f}%<br>"
        # annotation_text += f"• Training Range: {df_before_prediction['year_int'].min()}-{df_before_prediction['year_int'].max()}<br><br>"
    else:
        annotation_text = f"<b>{prediction_year} Prediction for {symbol_prediction}</b><br>"
        annotation_text += f"<b>Asset</b>: {predicted_value:,.2f} Billion VND<br><br>"
        annotation_text += f"<b>Training Data Statistics:</b><br>"
        train_data = df_before_prediction[feature_cols_prediction].values
        avg_growth = np.mean(np.diff(train_data[-5:, 0]) / train_data[-5:-1, 0] * 100)
        annotation_text += f"• Avg Growth Rate (last 5 years): {avg_growth:.2f}%<br>"
        # annotation_text += f"• Training Range: {df_before_prediction['year_int'].min()}-{df_before_prediction['year_int'].max()}<br>"

    # Create interactive Plotly chart
    fig = go.Figure()

    # Add historical data trace
    fig.add_trace(
        go.Scatter(
            x=df_symbol['year_int'],
            y=actual_data[:, 0],
            mode='lines+markers',
            name='Historical',
            line=dict(color='blue', width=2),
            marker=dict(size=8),
            hovertemplate='Year: %{x}<br>Asset: %{y:,.2f}B VND<extra></extra>'
        )
    )

    # fig.add_trace(
    #     go.Scatter(
    #         x=df_before_prediction['year_int'],
    #         y=actual_data[:len(df_before_prediction), 0],
    #         mode='lines+markers',
    #         name='Training Data',
    #         line=dict(color='purple', width=2),
    #         marker=dict(size=8),
    #         hovertemplate='Year: %{x}<br>Asset: %{y:,.2f}B VND<extra></extra>'
    #     )
    # )

    # Add prediction trace
    fig.add_trace(
        go.Scatter(
            x=future_years,
            y=[future_predictions[-1, 0]],
            mode='lines+markers',
            name='Predicted',
            line=dict(color='red', width=2, dash='dash'),
            marker=dict(color='red', size=12, symbol='square'),
            hovertemplate='Year: %{x}<br>Predicted: %{y:,.2f}B VND<extra></extra>'
        )
    )

    # Add actual point if available
    if actual_prediction_mask.any():
        fig.add_trace(
            go.Scatter(
                x=[prediction_year],
                y=[actual_value],
                mode='markers',
                name=f'Actual {prediction_year}',
                marker=dict(color='green', size=14, symbol='diamond'),
                hovertemplate='Year: %{x}<br>Actual: %{y:,.2f}B VND<extra></extra>'
            )
        )

    # Update layout
    fig.update_layout(
        title=f'{symbol_prediction} - Asset LSTM Prediction for {prediction_year}',
        xaxis_title='Year',
        yaxis_title='Asset (Billion VND)',
        height=600,
        showlegend=True,
        hovermode='x unified',
        template='plotly_white'
    )

    # Add annotation
    if annotation_text:
        fig.add_annotation(
            text=annotation_text,
            xref="paper", yref="paper",
            x=0.02, y=0.98,
            xanchor='left', yanchor='top',
            showarrow=False,
            bgcolor="rgba(255, 255, 255, 0.9)",
            bordercolor="black",
            borderwidth=1,
            font=dict(size=10),
            align='left'
        )

    fig.show()


NameError: name 'df_balance_sheet' is not defined

In [315]:
feature_cols = ['asset', 'debt', 'equity', 'cash', 'payable']

In [326]:
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import LeakyReLU

# Configuration
today = datetime.datetime.today()
training_end_year = 2022
look_back = today.year - training_end_year
# Prepare data
df_train = df_balance_sheet.copy()
df_train['year_int'] = df_train['year'].str.split('-').str[0].astype("int")

# Prepare training and validation data for all symbols
train_X, train_y = [], []
val_X, val_y = [], []

for symbol in df_train['symbol'].unique():
    if symbol != "FPT":
        continue
    df_symbol = df_train[df_train['symbol'] == symbol].copy()

    # Split: training data up to 2022, validation data from 2023 onwards
    train_mask = df_symbol['year_int'] <= training_end_year
    val_mask = df_symbol['year_int'] >= training_end_year + 1

    df_train_symbol = df_symbol[train_mask].sort_values('year_int')
    df_val_symbol = df_symbol[val_mask].sort_values('year_int')

    # Skip if insufficient training data
    if len(df_train_symbol) < look_back + 1:
        print(f"Insufficient data for {symbol}. Need at least {look_back + 1} years.")
        continue

    # Extract features and scale (fit scaler on training data only)
    train_data = df_train_symbol[feature_cols].values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_train_data = scaler.fit_transform(train_data)

    # Create training sequences
    X, y = create_sequences(scaled_train_data, look_back)

    if len(X) > 0:
        train_X.append(X)
        train_y.append(y)

    # Create validation sequences if validation data exists
    if len(df_val_symbol) > 0:
        # Use last look_back points from training + validation data
        combined_data = np.vstack([train_data[-look_back:], df_val_symbol[feature_cols].values])
        scaled_combined = scaler.transform(combined_data)

        X_val, y_val = create_sequences(scaled_combined, look_back)

        if len(X_val) > 0:
            val_X.append(X_val)
            val_y.append(y_val)

# Combine all sequences
X_train = np.vstack(train_X)
y_train = np.vstack(train_y)

print(f"Training data shape: X={X_train.shape}, y={y_train.shape}")
print(f"Training period: historical data to {training_end_year}")
print(f"Look-back window: {look_back} years")

# Prepare validation data if available
if len(val_X) > 0:
    X_val = np.vstack(val_X)
    y_val = np.vstack(val_y)
    print(f"Validation data shape: X={X_val.shape}, y={y_val.shape}")
    print(f"Validation period: 2023 to {today.year}")
    validation_data = (X_val, y_val)
else:
    print("No validation data available")
    validation_data = None

# Build LSTM model
model = Sequential([
    Input(shape=(look_back, len(feature_cols))),
    LSTM(50, activation='relu', return_sequences=True),
    # LeakyReLU(),
    GRU(50, activation='relu', return_sequences=True),
    Dropout(0.3),
    LSTM(50, activation='relu'),
    # LeakyReLU(),
    Dropout(0.3),
    Dense(len(feature_cols))
])

model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

# Train model with validation data
early_stopping = EarlyStopping(
    monitor='val_loss' if validation_data else 'loss',
    patience=20,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_data=validation_data,
    epochs=100,
    batch_size=32,
    verbose=1,
    callbacks=[early_stopping]
)

# Randomly shuffle data for train-validation split
indices = np.arange(len(X_train))
np.random.seed(42)
np.random.shuffle(indices)

split_idx = int(len(X_train) * (0.8))
train_indices = indices[:split_idx]
val_indices = indices[split_idx:]

X_train_split = X_train[train_indices]
y_train_split = y_train[train_indices]
X_val_split = X_train[val_indices]
y_val_split = y_train[val_indices]

# Evaluate on training and validation sets
train_loss = model.evaluate(X_train_split, y_train_split, verbose=0)
val_loss = model.evaluate(X_val_split, y_val_split, verbose=0)

# Save model
model.save('balancesheet_lstm_model_all_features_symbols.keras')
print("\n" + "=" * 80)
print("Model training completed and saved!")
print(f"Training samples: {len(X_train):,}")
if validation_data:
    print(f"Validation samples: {len(X_val):,}")
print("=" * 80)

print(f"\nModel Performance:")
print(f"  Training Loss (MSE): {train_loss:.6f}")
print(f"  Validation Loss (MSE): {val_loss:.6f}")

# Make predictions for visualization
y_train_pred = model.predict(X_train_split, verbose=0)
y_val_pred = model.predict(X_val_split, verbose=0)

# Calculate additional metrics
train_mse = mean_squared_error(y_train_split, y_train_pred)
val_mse = mean_squared_error(y_val_split, y_val_pred)

print(f"\nDetailed Metrics:")
print(f"  Training MSE: {train_mse:.6f}")
print(f"  Validation MSE: {val_mse:.6f}")

Training data shape: X=(18, 3, 5), y=(18, 5)
Training period: historical data to 2022
Look-back window: 3 years
Validation data shape: X=(2, 3, 5), y=(2, 5)
Validation period: 2023 to 2025


Model: "sequential_39"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_76 (LSTM)                  │ (None, 3, 50)          │        11,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_38 (GRU)                    │ (None, 3, 50)          │        15,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_76 (Dropout)            │ (None, 3, 50)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_77 (LSTM)                  │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_77 (Dropout)            │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 5)              │           255 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 46,955 (183.42 KB)

 Trainable params: 46,955 (183.42 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.2419 - val_loss: 1.4590
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.2392 - val_loss: 1.4503
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.2371 - val_loss: 1.4417
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.2351 - val_loss: 1.4327
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.2325 - val_loss: 1.4235
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.2306 - val_loss: 1.4147
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.2277 - val_loss: 1.4054
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.2250 - val_loss: 1.3956
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.2231 - val_loss: 1.3855
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.2191 - val_loss: 1.3748
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.2167 - val_loss: 1.3633
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.2132 - val_loss

In [327]:
# Predict asset of VNM symbol for user-specified year and visualize
from tensorflow.keras.models import load_model
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import datetime

# Load the saved model
loaded_model = load_model('balancesheet_lstm_model_all_features_symbols.keras')

# Configuration
# symbol_prediction = "VNM"
symbol_prediction = str(input("Enter the symbol you want to predict for 2023: ")).upper()
today = datetime.datetime.today()
look_back = 3
# FIXED: Use the same features as during training
feature_cols_prediction = ['asset', 'debt', 'equity', 'cash', 'payable']
# feature_cols_prediction = ['asset']

# User input for prediction year
prediction_year = 2023
future_years = [2023]

# Get symbol data
df_symbol = df_balance_sheet[df_balance_sheet['symbol'] == symbol_prediction].copy()
df_symbol['year_int'] = df_symbol['year'].str.split('-').str[0].astype(int)

# Check if we have enough data
if len(df_symbol) < look_back + 2:
    print(f"Insufficient data for {symbol_prediction}. Need at least {look_back + 2} years.")
else:
    # Prepare data - Use ALL historical data for fitting the scaler
    data = df_symbol[feature_cols_prediction].values

    # Normalize data using ALL data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    # Get data up to year before prediction for training
    df_before_prediction = df_symbol[df_symbol['year_int'] < prediction_year].copy().sort_values("year_int", ascending=True)
    data_before_prediction = df_before_prediction[feature_cols_prediction].values

    # Transform (not fit_transform) using the same scaler
    scaled_data_before_prediction = scaler.transform(data_before_prediction)

    # Predict for the specified year(s)
    last_sequence = scaled_data_before_prediction[-look_back:].copy()
    future_predictions = []

    # Calculate number of years to predict
    num_years_to_predict = prediction_year - df_before_prediction['year_int'].max()
    print(f"Number of years to predict: {num_years_to_predict}")

    for _ in range(num_years_to_predict):
        pred_input = last_sequence.reshape(1, look_back, len(feature_cols_prediction))
        pred = loaded_model.predict(pred_input, verbose=0)
        future_predictions.append(pred[0])
        last_sequence = np.vstack([last_sequence[1:], pred[0]])

    # Inverse transform predictions
    future_predictions = scaler.inverse_transform(np.array(future_predictions))

    # Prepare actual data
    actual_data = scaler.inverse_transform(scaled_data)

    # Calculate metrics if actual data is available for prediction year
    actual_prediction_mask = df_symbol['year_int'] == prediction_year
    predicted_value = future_predictions[-1, 0]  # Get the last prediction (target year)

    # Initialize variables for annotation
    annotation_text = ""
    actual_value = None

    if actual_prediction_mask.any():
        actual_values = actual_data[actual_prediction_mask.values].flatten()
        predicted_values = future_predictions[-1]

        mse = mean_squared_error(actual_values, predicted_values)
        rmse = np.sqrt(mse)

        # Calculate training data statistics
        train_data = df_before_prediction[feature_cols_prediction].values
        avg_growth = np.mean(np.diff(train_data[-5:, 0]) / train_data[-5:-1, 0] * 100)

        # Get comparison values
        actual_value = actual_data[actual_prediction_mask.values, 0][0]
        difference = predicted_value - actual_value
        percentage_diff = (difference / actual_value) * 100 if actual_value != 0 else 0

        # Build annotation text
        annotation_text = f"<b>{prediction_year} Prediction for {symbol_prediction}</b><br>"
        annotation_text += f"<b>Asset</b>: {predicted_value:,.2f} Billion VND<br><br>"

        annotation_text += f"<b>Comparison with Actual {prediction_year}:</b><br>"
        annotation_text += f"• Actual {prediction_year}: {actual_value:,.2f} Billion VND<br>"
        annotation_text += f"• Difference: {difference:,.2f} ({percentage_diff:+.2f}%)<br><br>"

        annotation_text += f"<b>Prediction Metrics:</b><br>"
        annotation_text += f"• MSE: {mse:,.2f}<br>"
        annotation_text += f"• RMSE: {rmse:,.2f}<br><br>"

        annotation_text += f"<b>Training Data Statistics:</b><br>"
        annotation_text += f"• Avg Growth Rate (last 5 years): {avg_growth:.2f}%<br>"
        # annotation_text += f"• Training Range: {df_before_prediction['year_int'].min()}-{df_before_prediction['year_int'].max()}<br><br>"
    else:
        annotation_text = f"<b>{prediction_year} Prediction for {symbol_prediction}</b><br>"
        annotation_text += f"<b>Asset</b>: {predicted_value:,.2f} Billion VND<br><br>"
        annotation_text += f"<b>Training Data Statistics:</b><br>"
        train_data = df_before_prediction[feature_cols_prediction].values
        avg_growth = np.mean(np.diff(train_data[-5:, 0]) / train_data[-5:-1, 0] * 100)
        annotation_text += f"• Avg Growth Rate (last 5 years): {avg_growth:.2f}%<br>"
        # annotation_text += f"• Training Range: {df_before_prediction['year_int'].min()}-{df_before_prediction['year_int'].max()}<br>"

    # Create interactive Plotly chart
    fig = go.Figure()

    # Add historical data trace
    fig.add_trace(
        go.Scatter(
            x=df_symbol['year_int'],
            y=actual_data[:, 0],
            mode='lines+markers',
            name='Historical',
            line=dict(color='blue', width=2),
            marker=dict(size=8),
            hovertemplate='Year: %{x}<br>Asset: %{y:,.2f}B VND<extra></extra>'
        )
    )

    # fig.add_trace(
    #     go.Scatter(
    #         x=df_before_prediction['year_int'],
    #         y=actual_data[:len(df_before_prediction), 0],
    #         mode='lines+markers',
    #         name='Training Data',
    #         line=dict(color='purple', width=2),
    #         marker=dict(size=8),
    #         hovertemplate='Year: %{x}<br>Asset: %{y:,.2f}B VND<extra></extra>'
    #     )
    # )

    # Add prediction trace
    fig.add_trace(
        go.Scatter(
            x=future_years,
            y=[future_predictions[-1, 0]],
            mode='lines+markers',
            name='Predicted',
            line=dict(color='red', width=2, dash='dash'),
            marker=dict(color='red', size=12, symbol='square'),
            hovertemplate='Year: %{x}<br>Predicted: %{y:,.2f}B VND<extra></extra>'
        )
    )

    # Add actual point if available
    if actual_prediction_mask.any():
        fig.add_trace(
            go.Scatter(
                x=[prediction_year],
                y=[actual_value],
                mode='markers',
                name=f'Actual {prediction_year}',
                marker=dict(color='green', size=14, symbol='diamond'),
                hovertemplate='Year: %{x}<br>Actual: %{y:,.2f}B VND<extra></extra>'
            )
        )

    # Update layout
    fig.update_layout(
        title=f'{symbol_prediction} - Asset LSTM Prediction for {prediction_year}',
        xaxis_title='Year',
        yaxis_title='Asset (Billion VND)',
        height=600,
        showlegend=True,
        hovermode='x unified',
        template='plotly_white'
    )

    # Add annotation
    if annotation_text:
        fig.add_annotation(
            text=annotation_text,
            xref="paper", yref="paper",
            x=0.02, y=0.98,
            xanchor='left', yanchor='top',
            showarrow=False,
            bgcolor="rgba(255, 255, 255, 0.9)",
            bordercolor="black",
            borderwidth=1,
            font=dict(size=10),
            align='left'
        )

    fig.show()


Number of years to predict: 1
